In [ ]:
import torch
import models.p2pnet
import numpy
import os
import argparse
import random
import ast
import seaborn

from torch.utils.data import DataLoader

from crowd_datasets import build_dataset
from engine import *
from models import build_model

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# load in models and dataset
result_dir = "results/updateset_highres_emb_FPN1_r2_c2_512ps/" 
model_dir =  "weights/best_mae.pth"
data_dir = "dataroot/updated_all_images"

# load in the arugments from the result di 
def load_args(filepath):
    with open(filepath, 'r') as f:
        namespace_str = f.read().strip()
    
    # Remove "Namespace(" from beginning and ")" from end
    params_str = namespace_str[10:-1]
    
    # Parse the parameters into a dictionary
    params = {}
    for param in params_str.split(','):
        if '=' in param:
            key, value = param.split('=', 1)
            key = key.strip()
            value = value.strip()
            # Handle different value types
            if value.lower() == 'true':
                value = True
            elif value.lower() == 'false':
                value = False
            elif value.lower() == 'none':
                value = None
            else:
                try:
                    value = float(value) if '.' in value else int(value)
                except ValueError:
                    # If not a number, remove quotes if present
                    value = value.strip("'\"")
                    # make list if brakets are present
                    if '[' in value: value = list(value)

            params[key] = value
    
    return argparse.Namespace(**params)

args = load_args(os.path.join(result_dir, "args.txt"))

# hard code some args
args.loss = ['labels', 'points']
args.multiclass = ['embryo']
args.ce_coef = [1.0]
args.dataset_file = "WORM_VAL"
args.gpu_id = 1

print(args) 

# load in model
model = build_model(args)
checkpoint = torch.load(os.path.join(result_dir, model_dir), map_location='cpu')
model.load_state_dict(checkpoint['model'])
model.to(device)
model.eval()


# import dataset
loading_data = build_dataset(args)

train_set, val_set = loading_data(
    data_dir,
    multiclass=args.multiclass,
    hsv=args.hsv,
    hse=args.hse,
    edges=args.edges,
    equalize=args.equalize,
)
# create the sampler used during training
sampler_val = torch.utils.data.SequentialSampler(val_set)

 
data_loader_val = DataLoader(
    val_set,
    1,
    sampler=sampler_val,
    drop_last=False,
    collate_fn=utils.collate_fn_crowd,
    num_workers=args.num_workers,
)

print(model)

Namespace(lr=0.0001, lr_backbone='1e-05', batch_size=3, weight_decay=0.0001, epochs=400, lr_drop=100, clip_max_norm=0.1, frozen_weights=None, pre_weights=None, backbone='vgg16_bn', num_classes=1, set_cost_class=1, set_cost_point=0.05, loss=['labels', 'points'], label_loss_coef=1, point_loss_coef=0.05, dense_loss_coef=1.0, count_loss_coef=10.0, distance_loss_coef=1.0, eos_coef=0.5, ce_coef=[1.0], map_res=16, gauss_kernel_res=21, row=2, line=2, dataset_file='WORM_VAL', data_root='dataroot/updated_all_images/', expname='updateset_highres_emb_FPN1_r2_c2_512ps', output_dir='./results/', multiclass=['embryo'], hsv=False, hse=False, edges=False, sharpness=True, scale=True, equalize=True, num_patches=4, patch_size=512, seed=42, resume=None, start_epoch=0, pointmatch=False, noreg=False, classifier=False, eval=False, num_workers=8, eval_freq=1, gpu_id=1)


Using cache found in /home/sposhiya/.cache/torch/hub/pytorch_vision_main
/home/sposhiya/.conda/envs/pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sposhiya/.conda/envs/pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


P2PNet(
  (backbone): Backbone_VGG(
    (body1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
    )
    (body2): Sequential(
      (0): MaxPool2d(ke

In [5]:
conv_out = {}
def set_hook(name):
    def hook(model, input, output):
        conv_out[name] = output.detach()
    return hook

model.classification.conv1.register_forward_hook(set_hook("class_conv_1"))
model.classification.conv2.register_forward_hook(set_hook("class_conv_2"))
model.classification.output.register_forward_hook(set_hook("class_out"))
model.regression.conv1.register_forward_hook(set_hook("reg_conv_1"))
model.regression.conv2.register_forward_hook(set_hook("reg_conv_2"))
model.regression.output.register_forward_hook(set_hook("reg_out"))

X,Y = next(iter(data_loader_val))
X = X.to(device)
output = model(X)

/home/sposhiya/.conda/envs/pytorch/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 4.50 GiB. GPU  has a total capacity of 79.22 GiB of which 792.88 MiB is free. Process 1227058 has 13.70 GiB memory in use. Process 1249482 has 7.12 GiB memory in use. Process 612467 has 28.38 GiB memory in use. Including non-PyTorch memory, this process has 29.21 GiB memory in use. Of the allocated memory 28.42 GiB is allocated by PyTorch, and 22.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Exception raised from malloc at /opt/conda/conda-bld/pytorch_1712608847532/work/c10/cuda/CUDACachingAllocator.cpp:1143 (most recent call first):
frame #0: c10::Error::Error(c10:

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.25 GiB. GPU  has a total capacity of 79.22 GiB of which 790.88 MiB is free. Process 1227058 has 13.70 GiB memory in use. Process 1249482 has 7.12 GiB memory in use. Process 612467 has 28.38 GiB memory in use. Including non-PyTorch memory, this process has 29.22 GiB memory in use. Of the allocated memory 28.42 GiB is allocated by PyTorch, and 22.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
len(conv_out)
print(list(conv_out.values())[0].size())

IndexError: list index out of range